In [14]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from Database.MongoDB_Connection import start_db, close_db
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import joblib

In [15]:
# Database connection method is used here in order to bring all the database documents over from the collection. The connection should already be made via MongoDB_Operations file. If not, the connection will be made now.


print(os.getcwd())
db = start_db()
print(db)
get_data = db.Finance_Data.find({
    "Ticker": {"$exists": True},
    "SMA": {"$exists": True},
    "EMA": {"$exists": True},
    "MACD": {"$exists": True},
    "RSI": {"$exists": True},
    "Success": {"$exists": True}
},
    {"_id":0})

df = pd.DataFrame(list(get_data))
print(type(df))

C:\Users\CRTITSS\Desktop\Projects\Backend_Content
['C964_Database', 'admin', 'config', 'local']
Test entry exists
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'C964_Database')
<class 'pandas.core.frame.DataFrame'>


In [16]:
# Separate the document fields from the "Success" target column

X = df[["Open", "High", "Low", "Volume", "Sentiment", "SMA", "EMA", "MACD", "RSI"]]
y = df["Success"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=471189)

In [17]:
# Train the model.

# Initialize the XGB model. Label encoder parameter is used to safeguard against a depracated functions with sci-kit learn. Eval metric is set to LogLoss to show how well the model performed. It will return a number between 0 - 1. 0 meaning it performed perfectly( which means overfitting likely occurred which is not a good thing). N_jobs refers to the number of parallel threads used to train. It is set to -1 to use all available threads to speed up training.
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_jobs=-1)

model.fit(X_train, y_train)

joblib.dump(model, "C964_model.joblib")

C:\Users\CRTITSS\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:58:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


['C964_model.joblib']

In [18]:
# Evaluate the model and print the results.

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f" Accuracy: {acc:.2f}")
print(classification_report(y_test, y_pred))

 Accuracy: 0.73
              precision    recall  f1-score   support

           0       0.74      0.76      0.75     21260
           1       0.73      0.71      0.72     19294

    accuracy                           0.73     40554
   macro avg       0.73      0.73      0.73     40554
weighted avg       0.73      0.73      0.73     40554



In [19]:
# Evaluate a random Ticker not in the database. The ticker can already exist in the database. This would be the most up-to-date information theoretically.

# import yfinance as yf
#
# data = yf.download("BTC", period="max", interval="1d", progress=False)

